In [16]:

import numpy as np
from sklearn.base import BaseEstimator
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from itertools import combinations
from collections import Counter

In [ ]:
class OVAclasificador(BaseEstimator):
   

    def __init__(self, C=1.0, solver='lbfgs', max_iter=1000):
        # Hiperparámetros (valores escalares)
        self.C = C
        self.solver = solver
        self.max_iter = max_iter
        # Diccionario donde guardaremos los clasificadores entrenados por etiqueta
        # Antes de Fit: {}
        # Después de Fit: {etiqueta: clf_entrenado, ...}
        self.modelos = {}
        # None antes de Fit; después será un numpy array 1D con etiquetas únicas
        self.clases_ = None

    def Fit(self, X, y):
        # Asegurar formatos numpy
        X = np.asarray(X)
        y = np.asarray(y)
        # self.clases_ guarda etiquetas únicas en orden ascendente
        # Ejemplo: array([0,1,2])
        self.clases_ = np.unique(y)
        # Reiniciar/limpiar modelos previos
        self.modelos = {}

        # Para cada clase entrenamos un clasificador binario
        for clase in self.clases_:
            # y_bin: vector de ceros/unos donde 1 indica la clase actual
            # Forma: (n_samples,)
            y_bin = (y == clase).astype(int)

            # Creamos el clasificador con los hiperparámetros guardados
            clf = LogisticRegression(C=self.C, solver=self.solver, max_iter=self.max_iter)

            # fit entrena internamente y guarda coeficientes (clf.coef_), intercept (clf.intercept_)
            # Después de fit: clf tiene atributos como coef_, intercept_, n_iter_ (según solver)
            clf.fit(X, y_bin)

            # Guardamos el clasificador entrenado en el diccionario
            # Key: etiqueta (por ejemplo 0), Value: clf (entrenado)
            self.modelos[clase] = clf

        # Devolvemos self para permitir encadenamiento (estilo sklearn)
        return self

    def Predict(self, X):
        # X: (n_query, n_features)
        X = np.asarray(X)
        if self.clases_ is None:
            raise RuntimeError("El clasificador no está entrenado. Ejecutar Fit primero.")
        puntuaciones = []

        for clase in self.clases_:
            clasificador = self.modelos[clase]
            if hasattr(clasificador, "predict_proba"):
                # predict_proba(X) -> array (n_query, 2)
                # [:,1] => probabilidad de la etiqueta positiva (1)
                probs = clasificador.predict_proba(X)[:, 1]
                # probs: array (n_query,)
                puntuaciones.append(probs)

            # Si no hay predict_proba, pero hay decision_function, lo convertimos a prob
            elif hasattr(clasificador, "decision_function"):
                # decision_function(X) -> array (n_query,) con logits reales
                df = clasificador.decision_function(X)
                # transformar logits a [0,1] con la función sigmoide
                probs = 1 / (1 + np.exp(-df))
                puntuaciones.append(probs)

            else:
                # Último recurso: usar predict (0/1). No es ideal como "confianza" pero sirve.
                preds = clasificador.predict(X)
                # convertimos a float para compatibilidad con stacking
                puntuaciones.append(preds.astype(float))

        # Apilar columnas -> matriz (n_query, n_clases)
        puntuaciones = np.vstack(puntuaciones).T
        # idx: para cada muestra, índice de la clase con mayor puntuación
        indice = np.argmax(puntuaciones, axis=1)
        # Devolver etiquetas originales según self.clases_
        # Por ejemplo, si indice[0]==2 y self.clases_[2]==2 -> primera muestra predicha como etiqueta 2
        return self.clases_[indice]

In [ ]:
class OVOclasificador(BaseEstimator):
    """One-vs-One: para cada par de clases entrenamos un clasificador binario.

    Detalle de variables y estados:
    - self.clases_: etiquetas únicas encontradas en y (shape: (n_clases,)).
    - self.modelos: dict con claves (clase_a, clase_b) y valores = clasificador entrenado
      solo con muestras de esas dos clases.

    En Fit:
    - mascara: boolean array (n_samples,) True para muestras pertenecientes a la pareja
    - X_par: subarray de X con solo filas seleccionadas por mascara (n_pair, n_features)
    - y_par: subvector de y correspondiente a X_par (n_pair,)
    - y_bin: etiqueta binaria para el par (1->clase1, 0->clase2)

    En Predict:
    - votos: matriz entera (n_query, n_clases) con conteo de votos de cada clasificador

    Elección final: la clase con más votos para cada muestra.
    """

    def __init__(self, C=1.0, solver='lbfgs', max_iter=1000):
        self.C = C
        self.solver = solver
        self.max_iter = max_iter
        self.modelos = {}
        self.clases_ = None

    def Fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)
        # etiquetas unicas (por ejemplo array([0,1,2]))
        self.clases_ = np.unique(y)
        self.modelos = {}

        # Iterar sobre pares (clase1, clase2)
        for i, clase1 in enumerate(self.clases_):
            for clase2 in self.clases_[i + 1:]:
                # mascara: True para filas donde y==clase1 o y==clase2
                mascara = np.logical_or(y == clase1, y == clase2)
                # X_par: (n_pair, n_features)
                X_par = X[mascara]
                # y_par: (n_pair,)
                y_par = y[mascara]
                # y_bin: 1 si y_par==clase1, 0 si y_par==clase2
                y_bin = np.where(y_par == clase1, 1, 0)

                # Entrenar clasificador binario sólo con las dos clases
                clasificador = LogisticRegression(C=self.C, solver=self.solver, max_iter=self.max_iter)
                clasificador.fit(X_par, y_bin)

                # Guardar el clasificador con clave (clase1, clase2)
                self.modelos[(clase1, clase2)] = clasificador

        return self

    def Predict(self, X):
        X = np.asarray(X)
        if self.clases_ is None:
            raise RuntimeError("El clasificador no está entrenado. Ejecutar Fit primero.")

        # votos: matriz (n_query, n_clases), inicializada a ceros
        votos = np.zeros((X.shape[0], len(self.clases_)), dtype=int)

        # Cada clasificador de la pareja vota por una de las dos clases
        for (clase1, clase2), clasificador in self.modelos.items():
            # preds: vector (n_query,) con valores 0 o 1
            preds = clasificador.predict(X)
            # Obtener índices de clase1 y clase2 en self.clases_
            idx1 = int(np.where(self.clases_ == clase1)[0][0])
            idx2 = int(np.where(self.clases_ == clase2)[0][0])

            # Si un clasificador predice 1 -> suma voto a idx1, si 0 -> a idx2
            votos[preds == 1, idx1] += 1
            votos[preds == 0, idx2] += 1

        # Resultado final: para cada muestra, tomar la clase con más votos
        indices_pred = np.argmax(votos, axis=1)
        return self.clases_[indices_pred]

In [ ]:
class SoftmaxRegression(BaseEstimator):
  

    def __init__(self, C=None, lr=0.1, max_iter=1000, tol=1e-5, reg=None, verbose=False, **kwargs):
        if reg is None:
            if C is None:
                self.reg = 1e-3
            else:
                self.reg = 1.0 / C
        else:
            self.reg = reg
        self.lr = lr
        self.max_iter = max_iter
        self.tol = tol
        self.verbose = verbose
        self.W = None
        self.classes_ = None

    def _one_hot(self, y_idx, K):
        n = y_idx.shape[0]
        Y = np.zeros((n, K))
        Y[np.arange(n), y_idx] = 1
        return Y

    def Fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)
        self.classes_ = np.unique(y)
        K = len(self.classes_)
        class_to_idx = {c: i for i, c in enumerate(self.classes_)}
        y_idx = np.vectorize(class_to_idx.get)(y)

        n, d = X.shape
        Xb = np.hstack([np.ones((n, 1)), X])

        self.W = np.zeros((d + 1, K))

        Y = self._one_hot(y_idx, K)

        for it in range(self.max_iter):
            scores = Xb.dot(self.W)
            scores -= scores.max(axis=1, keepdims=True)
            exp_scores = np.exp(scores)
            probs = exp_scores / exp_scores.sum(axis=1, keepdims=True)

            grad = - (Xb.T.dot(Y - probs)) / n
            reg_term = self.reg * np.vstack([np.zeros((1, K)), self.W[1:, :]])
            grad += reg_term

            W_old = self.W.copy()
            self.W -= self.lr * grad

            diff = np.linalg.norm(self.W - W_old)
            if self.verbose and (it % 100 == 0 or it == self.max_iter - 1):
                loss = -np.mean(np.sum(Y * np.log(probs + 1e-15), axis=1)) + 0.5 * self.reg * np.sum(self.W[1:, :] ** 2)
                print(f"it={it} loss={loss:.6f} ||dW||={diff:.6e}")
            if diff < self.tol:
                break
        return self

    def Predict(self, X):
        if self.W is None:
            raise RuntimeError("El clasificador no está entrenado. Ejecutar Fit primero.")
        X = np.asarray(X)
        n = X.shape[0]
        Xb = np.hstack([np.ones((n, 1)), X])
        scores = Xb.dot(self.W)
        scores -= scores.max(axis=1, keepdims=True)
        exp_scores = np.exp(scores)
        probs = exp_scores / exp_scores.sum(axis=1, keepdims=True)
        idx = np.argmax(probs, axis=1)
        return self.classes_[idx]

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import time

data = load_iris()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

ova = OVAclasificador()
ovo = OVOclasificador()
try:
    sm = SoftmaxRegression(lr=0.5, max_iter=2000, tol=1e-7, verbose=False, reg=1e-3)
except TypeError:
    sm = SoftmaxRegression(C=1.0, solver='lbfgs', max_iter=2000)

models = [('OvA', ova), ('OvO', ovo), ('Softmax', sm)]

results = {}

for name, model in models:
    t0 = time.time()
    model.Fit(X_train_s, y_train)
    y_pred = model.Predict(X_test_s)
    t1 = time.time()
    acc = accuracy_score(y_test, y_pred)
    results[name] = {'accuracy': acc, 'time_s': t1 - t0, 'y_pred': y_pred}
    print(f"{name}: accuracy={acc:.4f} time={t1-t0:.3f}s")
    print(classification_report(y_test, y_pred, target_names=data.target_names))
    print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))
    print("-" * 60)

print("Resumen:")
for name, info in results.items():
    print(f"{name}: acc={info['accuracy']:.4f} time={info['time_s']:.3f}s")

OvA: accuracy=0.8444 time=0.025s
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        15
  versicolor       0.79      0.73      0.76        15
   virginica       0.75      0.80      0.77        15

    accuracy                           0.84        45
   macro avg       0.85      0.84      0.84        45
weighted avg       0.85      0.84      0.84        45

Confusion matrix:
 [[15  0  0]
 [ 0 11  4]
 [ 0  3 12]]
------------------------------------------------------------
OvO: accuracy=0.9111 time=0.010s
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        15
  versicolor       0.82      0.93      0.88        15
   virginica       0.92      0.80      0.86        15

    accuracy                           0.91        45
   macro avg       0.92      0.91      0.91        45
weighted avg       0.92      0.91      0.91        45

Confusion matrix:
 [[15  0  0]
 [ 0 14  1]
 [ 0  3 12]]